In [23]:
import neat
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import graphviz

excel_file = "F:\Thesis\salesdaily-kaggle.xlsx"
sheet_name = "Worksheet"
column_name = "M01AB"

df = pd.read_excel(excel_file, sheet_name=sheet_name)
DATA=df.drop(columns=['datum','Year','Hour',])


""" print(DATA.head(50)) """
""" plt.plot(DATA['Month'],DATA['M01AB'],'o-')
plt.show() """

" plt.plot(DATA['Month'],DATA['M01AB'],'o-')\nplt.show() "

This will be for the first medicne

In [24]:
med1data= DATA[['M01AB','Month','Weekday Name']].copy()
label_encoder = LabelEncoder()
med1data['Weekday Name'] = label_encoder.fit_transform(med1data['Weekday Name'])
""" print(med1data.head()) """

for i in range (1,8):
    med1data[f'Lag{i}']=med1data['M01AB'].shift(i)
med1data.dropna(inplace=True)
""" print(med1data.head(100)) """


' print(med1data.head(100)) '

In [25]:
x=med1data.drop(columns='M01AB')
y=med1data['M01AB']
x_np= x.values
y_np=y.values

x_train,x_test,y_train,y_test= train_test_split(x_np,y_np,train_size=.8,random_state=42)
""" print(x_train,x_test) """
""" x_test,x_val, y_test, y_val=train_test_split(x_train,y_train,train_size=.5,random_state=42) """
""" print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)
print(y_test.shape) """

""" print(x_train.info()) """




' print(x_train.info()) '

In [26]:
config_file="F:\Thesis\SP maam code snippets\[NEAT].txt"
config= neat.Config(neat.DefaultGenome,neat.DefaultReproduction,neat.DefaultSpeciesSet,neat.DefaultStagnation,config_file)

p = neat.Population(config)
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)




	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_conn

In [27]:
def visualize(best_genome, config):
    node_names = {-1: 'Input', -2: 'Output'}  # Define names for input and output nodes
    for node_id, node in best_genome.nodes.items():
        node_names[node_id] = str(node_id)  # Assign names to other nodes

    # Create a graph using graphviz
    dot = graphviz.Digraph(format='png')
    for node_id, node in best_genome.nodes.items():
        if node_id in [-1, -2]:  # Check if the node is an input or output node
            color = 'lightblue' if node_id == -1 else 'lightcoral'
            dot.node(str(node_id), label=node_names[node_id], style='filled', fillcolor=color)
        else:
            color = 'lightgray'
            dot.node(str(node_id), label=node_names[node_id], style='filled', fillcolor=color)

    for conn_key, conn in best_genome.connections.items():
        if conn.enabled:
            in_node_id, out_node_id = conn_key
            dot.edge(str(in_node_id), str(out_node_id), label=str(round(conn.weight, 2)))

    # Save the graph to a file
    dot.render('neural_network', view=True)


In [28]:




# Define the fitness function for evaluating genomes
def evaluate_fitness(genomes, config, x_train, y_train):
    fitness_values = []
    kf=KFold(n_splits=5,shuffle=True,random_state=42)
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        fold_fitness= []
        
        for train_index, val_index in kf.split(x_np):
            x_train,x_val=x.iloc[train_index],x.iloc[val_index]
            y_train,y_val=y.iloc[train_index],y.iloc[val_index]
            x_train_np=x_train.values
            y_train_np=y_train.values
            x_val_np=x_val.values
            y_val_np=y_val.values
            predictions=[]
        # Make predictions on the training data
            for input_data in x_train_np:
                prediction = net.activate(input_data)
                predictions.append(prediction)
            fitness=calculate_fitness(predictions,y_train_np)
            fold_fitness.append(fitness)
        # Calculate fitness score based on predictions and target values using MSE
        
        genome.fitness = np.mean(fold_fitness)
        fitness_values.append(genome.fitness)
        
    # Calculate the range of fitness values


def calculate_fitness(predictions, targets):
    mse = np.mean((np.array(predictions) - np.array(targets)) ** 2)
    return 1 / (1 + mse)



# Assuming you have defined your NEAT population size
population_size = 150
best_genome=None
best_fitness=-1

# Run NEAT algorithm for a specified number of generations
num_generations = 50
for i in range(num_generations):
    # Evaluate fitness for the current generation
    winner = p.run(lambda genomes, config: evaluate_fitness(genomes, config, x_train, y_train), 1)
    # Filter out genomes with None fitness values
    valid_genomes = [genome for genome in p.population.values() if genome.fitness is not None]

# Get the best genome from the valid genomes
    gen_best_genome = max(valid_genomes, key=lambda x: x.fitness)
    
    if gen_best_genome.fitness>best_fitness:
        best_genome=gen_best_genome
        best_fitness=gen_best_genome.fitness
visualize(best_genome,config)
# After training, you can use the best genome to make predictions on the test data
best_net = neat.nn.FeedForwardNetwork.create(best_genome, config)
test_predictions = [best_net.activate(input_data) for input_data in x_test]

# Evaluate the performance of the model using test data
test_fitness = calculate_fitness(test_predictions, y_test)
print("Test Fitness:", test_fitness)





 ****** Running generation 0 ****** 

Population's average fitness: 0.03547 stdev: 0.00499
Best fitness: 0.04034 - size: (2, 10) - species 14 - id 14
Average adjusted fitness: 0.006
Mean genetic distance 5.192, standard deviation 2.117
Population of 450 members in 168 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     3      0.0    0.011     0
     2    0     3      0.0    0.000     0
     3    0     3      0.0    0.011     0
     4    0     4      0.0    0.011     0
     5    0     2      0.0    0.000     0
     6    0     3      0.0    0.002     0
     7    0     2      0.0    0.000     0
     8    0     3      0.0    0.000     0
     9    0     3      0.0    0.003     0
    10    0     4      0.0    0.000     0
    11    0     3      0.0    0.010     0
    12    0     4      0.0    0.008     0
    13    0     2      0.0    0.000     0
    14    0     2      0.0    0.011     0
    15    0     2      0.0    0.000     0
    16 

In [29]:
r_value = r2_score(y_test, test_predictions)
print("R-value:", r_value)

# Calculate RMSE (root mean squared error)
rmse_value = mean_squared_error(y_test, test_predictions, squared=False)
print("RMSE:", rmse_value)

R-value: -0.0020527994308943143
RMSE: 2.8137828824930873
